# Step38, 형상 변환 함수

이번에는 원소별로 계산하지 않는 함수에 대해 살펴본다.  
- 텐서의 형상을 변환하는 reshape 함수  
- 행렬을 전치하는 transpose 함수  

두 함수 모두 텐서의 형상을 바꾸는 함수이다. 

## 38.1 reshape 함수 구현 

먼저 텐서의 형상을 바꾸는 함수를 구현해본다.  
사전 준비로 넘파이의 reshape 함수의 사용법을 확인해본다.

In [12]:
import numpy as np 

x = np.array([[1,2,3],[4,5,6]])
y = np.reshape(x, (6,))
print(y)
print()
y = np.reshape(x, (2,3))
print(y)
print()
y = np.reshape(x, (3,-1))
print(y)

[1 2 3 4 5 6]

[[1 2 3]
 [4 5 6]]

[[1 2]
 [3 4]
 [5 6]]


텐서의 원소 수는 같고 형상만 바뀐다.  
이제 DeZero 버전의 reshape 함수를 구현해 본다. 문제는 역전파를 어떻게 구현하느냐이다.

변수의 데이터와 기울기의 형상이 일치하는지 확인해야한다.  
예: x가 Variable 인스턴스일 때 x.data.shape == x.grad.shape를 만족할 수 있도록 역전파를 구현해야한다.

reshape 함수는 단순히 형상만 변환한다. 구체적인 계산은 아무것도 하지 않는다.  
따라서 역전파는 출력 쪽에서 전해지는 기울기에 아무런 손도 대지 않고 입력 쪽으로 흘려보내준다.

그러나 기울기의 형상이 입력의 형상과 같아지도록 변환한다.

In [13]:
# functions.py
import dezero.functions as F 
from dezero import Variable, Function

class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape  # 클래스를 초기화할 때 변형 목표가 되는 형상을 shape 인수로 받는다.
    
    def forward(self, x):   # 순전파는 넘파이의 reshape 함수를 사용하여 형상을 변환한다.
        self.x_shape = x.shape  # 이때 입력 x의 shape을 기억해둔다.
        y = x.reshape(self.shape)
        return y 
    
    def backward(self, gy):     # 역전파에서 입력 shape(self.x_shape)으로 변환할 수 있다.
        return reshape(gy, self.x_shape)

In [14]:
# functions.py
from dezero.core import as_variable

def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)

# reshape 함수가 Variable 인스턴스를 반환함을 보장하기 위해 as_Variable 함수를 사용하여 Variable 인스턴스로 변환한다.

방금 구현한 reshape 함수를 사용해본다.

In [15]:
import numpy as np 
from dezero import Variable
import dezero.functions as F 

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.reshape(x, (6,))
print(y)
y.backward(retain_grad=True)
print(x.grad)

variable([1 2 3 4 5 6])
variable([[1 1 1]
          [1 1 1]])


reshape 함수를 사용하여 shape을 변환시켰다.  
그리고 y.backward(retain_grad=True)를 수행하여 x의 기울기를 구한다. 이 과정에서 y의 기울기도 자동으로 채워진다.  
채워진 기울기의 shape은 y와 같고(y.grad.shape == y.shape) 원소는 모두 1로 이루어진 텐서이다.

## 38.2 Variable에서 reshape사용하기


In [16]:
x = np.random.rand(1,2,3)

y = x.reshape((2,3))    # 튜플로 받기 
y = x.reshape([2,3])    # 리스트로 받기 
y = x.reshape(2,3)      # 인수를 그대로(풀어서) 받기 


보는것과 같이 reshape를 ndarray 인스턴스의 메서드로 사용할 수 있다.  
또한 x.reshape(2,3)과 같이 가변 인수도 받는다.  

DeZero에서도 이와 같은 용법을 제공해본다.
Variable 클래스에서 다음 코드를 추가한다.

~~~python 
import dezero 

class Variable:
    # ... 생략 ....
    
    def reshape(self, *shape):
        if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
            shape = shape[0]
        return dezero.functions.reshape(self, shape)
~~~

In [17]:
x = Variable(np.random.randn(1,2,3))
y = x.reshape((2,3))
y = x.reshape(2,3)

## 38.3 행렬의 전치 

In [18]:
x = np.array([[1,2,3],[4,5,6]])
y = np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


텐서의 원소 자체는 그대로이고 shape만 바뀐다.  
따라서 역전파에서는 출력 쪽에서 전해지는 기울기의 형상만 변경한다.  
순전파때와 '반대' 형태

dezero의 functions.py에 아래 코드 추가 

~~~python
class Transpose(Function):
    def forward(self, x):
        y = np.transpose(x)
        return y 
    
    def backward(self, gy):
        gx = transpose(gy)
        return gx 

def transpose(x):
    return Transpose()(x)

~~~

In [19]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.transpose(x)
y.backward()
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


transpose 함수를 사용하여 계산할 수 있으며 역전파도 제대로 이루어진다.

이어서 Variable 인스턴스에서도 transpose 함수를 사용할 수 있도록 다음 코드를 추가한다.

~~~python 
class Variable:
    # ... 생략 ...
    
    def transpose(self):
        return dezero.functions.transpose(self)
    
    @property
    def T(self):
        return dezero.functions.transpose(self)
~~~

첫번째 transpose는 '인스턴스 메서드'로 이용하기 위한 코드   
두번째 T에서 @property 데코레이터는 '인스턴스 변수'로 사용할 수 있게 해주는 데코레이터 

In [20]:
x = Variable(np.random.rand(2,3))
y = x.transpose()
y = x.T

In [21]:
A,B,C,D = 1,2,3,4
x = np.random.rand(A,B,C,D)
print(x)
print('--'*60)
y = x.transpose(1,0,3,2)
print(y)

[[[[0.94675345 0.06813661 0.71280146 0.21636013]
   [0.02434364 0.78739578 0.30817433 0.75960637]
   [0.41693737 0.42448872 0.6645393  0.32633775]]

  [[0.41024571 0.31967415 0.86505488 0.70669249]
   [0.35600245 0.01436504 0.08907842 0.14806674]
   [0.17050307 0.40150172 0.62903316 0.77640139]]]]
------------------------------------------------------------------------------------------------------------------------
[[[[0.94675345 0.02434364 0.41693737]
   [0.06813661 0.78739578 0.42448872]
   [0.71280146 0.30817433 0.6645393 ]
   [0.21636013 0.75960637 0.32633775]]]


 [[[0.41024571 0.35600245 0.17050307]
   [0.31967415 0.01436504 0.40150172]
   [0.86505488 0.08907842 0.62903316]
   [0.70669249 0.14806674 0.77640139]]]]


In [22]:
W1 = Variable(0.01 * np.random.rand(1,10))
print(W1.shape)
b1 = Variable(np.zeros(10))
print(b1.shape)
W2 = Variable(0.01 * np.random.rand(10,0))
print(W2.shape)
b2 = Variable(np.zeros(0))
print(b2.shape)

(1, 10)
(10,)
(10, 0)
(0,)
